# Advanced RAG 01: Small to Big

### Child-Parent RecursiveRetriever and Sentence Window Retrieval with LlamaIndex

Sources:
- https://docs.llamaindex.ai/en/stable/examples/retrievers/recursive_retriever_nodes.html
- https://docs.llamaindex.ai/en/latest/examples/node_postprocessor/MetadataReplacementDemo.html

In [1]:
! pip install -U llama_hub llama_index braintrust autoevals pypdf pillow transformers torch torchvision

  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.16.0+cu121
    Uninstalling torchvision-0.16.0+cu121:
      Successfully uninstalled torchvision-0.16.0+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.2.0 which is incompatible.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.


In [18]:
import os
os.environ["OPENAI_API_KEY"] = ""

# Basic RAG Review

In [6]:
from pathlib import Path
from llama_hub.file.pdf.base import PDFReader
from llama_index.response.notebook_utils import display_source_node
from llama_index.retrievers import RecursiveRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index import VectorStoreIndex, ServiceContext
from llama_index.llms import OpenAI
import json

### Step 1: Loading Documents

In [7]:
loader = PDFReader()
docs0 = loader.load_data(file=Path("llama2.pdf"))

In [8]:
docs0[0]

Document(id_='d335aed7-2244-4285-85bb-7ca7795386c9', embedding=None, metadata={'page_label': '1', 'file_name': 'llama2.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='018b304d342200869f05b9e3bcb41cf91a368cd743da48c666fb67110a4a2c1f', text='Llama 2 : Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗Louis Martin†Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou\nHakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev\nPunit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana Liskovich\nYinghai Lu Yuning Mao Xavier Martinet Todor Mihaylov Pushkar Mishra\nIgor Molybog Yixin Nie Andrew Poulton Jeremy Reizenstein R

In [10]:
from llama_index import Document

doc_text = "\n\n\n".join([d.get_content() for d in docs0]) # into single text
docs = [Document(text=doc_text)]

### Step 2: Parsing Documents into Text Chunks (Nodes)

```
SimpleNodeParser에서의 chunk 전략
1. 먼저 하나의 text를 여러 개의 split으로 쪼갠다.
  split 쪼개는 원리  : 먼저 paragraph로 쪼개지면 종료, 아니라면 sentence로 쪼개기, 아니라면 정규식으로 쪼개기, 아니라면 word로 쪼개기, 아니라면 char로 쪼개기
    1. split by paragraph separator
    2. split by chunking tokenizer (default is nltk sentence tokenizer)
    3. split by second chunking regex (default is "[^,\.;]+[,\.;]?")
    4. split by default separator (" ")

2. split들을 chunk_size를 고려해서 merge 한다.
  이를 통해서 하나의 split으로 들어갈 것이 여러 chunk로 fragment되는 것을 나름 방지한다.
  여기서 chunk_size 는 char 단위가 아니라 token_size로 봐야 한다.

```

In [38]:
from llama_index.node_parser import SimpleNodeParser
from llama_index.schema import IndexNode

node_parser = SimpleNodeParser.from_defaults(chunk_size=1024)
base_nodes = node_parser.get_nodes_from_documents(docs)
for idx, node in enumerate(base_nodes):
  node.id_ = f"node-{idx}"  # replace id_ from hash-like to number
len(base_nodes)

95

In [39]:
base_nodes[0]

TextNode(id_='node-0', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='a7705984-64ba-4a8b-8d84-2196a137671f', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='398cb8798614de7e8a14cfff4b34f87a9f979d4dfaac81358c311ca67a659fe0'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='a3bf5ad9-a072-441c-a9be-7fcf8c4d6894', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='820de41da4f942046105d6d061c52e7e04429023a554953b03b420aaebdb53ae')}, hash='b203c05456cf61d73bf4a69ecb61de212829d2a81f128ed71dc7612ec85ece86', text='Llama 2 : Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗Louis Martin†Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao V

### Step 3: Select Embedding Model and LLM

In [19]:
from llama_index.embeddings import resolve_embed_model

embed_model = resolve_embed_model("local:BAAI/bge-small-en")
llm = OpenAI(model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### Step 4: Create Index, retriever, and query engine

In [13]:
base_index = VectorStoreIndex(base_nodes, service_context=service_context)
base_retriever = base_index.as_retriever(similarity_top_k=2)

In [14]:
retrievals = base_retriever.retrieve(
    "Can you tell me about the key concepts for safety finetuning"
)

for n in retrievals:
    display_source_node(n, source_length=1500)

**Node ID:** node-26<br>**Similarity:** 0.8541543380274644<br>**Text:** TruthfulQA ↑ToxiGen ↓
MPT7B 29.13 22.32
30B 35.25 22.61
Falcon7B 25.95 14.53
40B 40.39 23.44
Llama 17B 27.42 23.00
13B 41.74 23.08
33B 44.19 22.57
65B 48.71 21.77
Llama 27B 33.29 21.25
13B 41.86 26.10
34B 43.45 21.19
70B 50.18 24.60
Table 11: Evaluation of pretrained LLMs on automatic safety benchmarks. For TruthfulQA, we present the
percentageofgenerationsthatarebothtruthfulandinformative(thehigherthebetter). ForToxiGen,we
present the percentage of toxic generations (the smaller, the better).
Benchmarks give a summary view ofmodel capabilities and behaviors that allow us to understand general
patternsinthemodel,buttheydonotprovideafullycomprehensiveviewoftheimpactthemodelmayhave
onpeopleorreal-worldoutcomes;thatwouldrequirestudyofend-to-endproductdeployments. Further
testing and mitigation should be done to understand bias and other social issues for the specific context
in which a system may be deployed. For this, it may be necessary to test beyond the groups available in
theBOLDdataset(race,religion,andgender). AsLLMsareintegratedanddeployed,welookforwardto
continuing research that will amplify their potential for positive impact on these important social issues.
4.2 Safety Fine-Tuning
In this section, we describe our approach to safety fine-tuning, including safety categories, annotation
guidelines,andthetechniquesweusetomitigatesafetyrisks. Weemployaprocesssimilartothegeneral
fine-tuning methods as described in Section 3, with some notable differences related to safet...<br>

**Node ID:** node-27<br>**Similarity:** 0.8464193261082225<br>**Text:** advice). The attackvectors exploredconsist ofpsychological manipulation(e.g., authoritymanipulation),
logic manipulation (e.g., false premises), syntactic manipulation (e.g., misspelling), semantic manipulation
(e.g., metaphor), perspective manipulation (e.g., role playing), non-English languages, and others.
Wethendefinebestpracticesforsafeandhelpfulmodelresponses: themodelshouldfirstaddressimmediate
safetyconcernsifapplicable,thenaddressthepromptbyexplainingthepotentialriskstotheuser,andfinally
provide additional information if possible. We also ask the annotators to avoid negative user experience
categories (see Appendix A.5.2). The guidelines are meant to be a general guide for the model and are
iteratively refined and revised to include newly identified risks.
4.2.2 Safety Supervised Fine-Tuning
InaccordancewiththeestablishedguidelinesfromSection4.2.1,wegatherpromptsanddemonstrations
ofsafemodelresponsesfromtrainedannotators,andusethedataforsupervisedfine-tuninginthesame
manner as described in Section 3.1. An example can be found in Table 5.
The annotators are instructed to initially come up with prompts that they think could potentially induce
themodel toexhibit unsafebehavior, i.e.,perform redteaming, asdefined bythe guidelines. Subsequently,
annotators are tasked with crafting a safe and helpful response that the model should produce.
4.2.3 Safety RLHF
Weobserveearlyinthedevelopmentof Llama 2-Chat thatitisabletogeneralizefromthesafedemonstrations
insupervisedfine-t...<br>

In [20]:
query_engine_base = RetrieverQueryEngine.from_args(
    base_retriever, service_context=service_context
)

response = query_engine_base.query(
    "Can you tell me about the key concepts for safety finetuning"
)
print(str(response))

The key concepts for safety fine-tuning include supervised safety fine-tuning, safety RLHF (Reinforcement Learning from Human Feedback), and safety context distillation. 

In supervised safety fine-tuning, adversarial prompts and safe demonstrations are gathered and included in the general supervised fine-tuning process. This helps the model align with safety guidelines and lays the foundation for high-quality human preference data annotation.

Safety RLHF involves integrating safety in the general RLHF pipeline. This includes training a safety-specific reward model and gathering more challenging adversarial prompts for rejection sampling style fine-tuning and PPO (Proximal Policy Optimization) optimization.

Safety context distillation is the final step in safety fine-tuning. It involves generating safer model responses by prefixing a prompt with a safety preprompt, such as "You are a safe and responsible assistant." The model is then fine-tuned on the safer responses without the prep

# Chunk References: Smaller Child Chunks Referring to Bigger Parent Chunk

In [25]:
sub_chunk_sizes = [128, 256, 512]
sub_node_parsers = [  # overlap은 20%
    SimpleNodeParser.from_defaults(chunk_size=c, chunk_overlap=int(c*0.2)) for c in sub_chunk_sizes
]

all_nodes = []
for base_node in base_nodes:  # 기존 larger chunk(1024)로 된 node들에 대해서 smaller chunk로 만든다.
  for n in sub_node_parsers:
    sub_nodes = n.get_nodes_from_documents([base_node]) # text node
    sub_inodes = [  # index node를 이용해서 smaller node 가 larger node를 pointing하게 한다.
        IndexNode.from_text_node(sn, base_node.node_id) for sn in sub_nodes
    ]
    all_nodes.extend(sub_inodes)

  # also add original node to node
  original_inode = IndexNode.from_text_node(base_node, base_node.node_id)
  all_nodes.append(original_inode)

all_nodes_dict = {n.node_id: n for n in all_nodes}
len(all_nodes_dict)

1648

In [36]:
all_nodes[0]

IndexNode(id_='7b1b798d-e4f6-4ea7-9073-303bfc4a6dc3', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='node-0', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='b203c05456cf61d73bf4a69ecb61de212829d2a81f128ed71dc7612ec85ece86'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='c59ae255-48f5-465d-88d4-73fc3e764bdd', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='c1cd7a3cd4e6b6f0bf126aec76d521214773696e2cc43ab2e814cffbb76582cf')}, hash='846d3ce8d8b3f2b35dce5dc8a696ccbded7b43b16619f156e69bcba19c7706e9', text='Llama 2 : Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗Louis Martin†Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Veda

In [40]:
vector_index_chunk = VectorStoreIndex(  # 100개에 2분 소요, 1600개면 30여분
    all_nodes, service_context=service_context
)
vector_retriever_chunk = vector_index_chunk.as_retriever(similarity_top_k=2)

In [ ]:
retriever_chunk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_chunk},
    node_dict=all_nodes_dict,
    verbose=True,
)

nodes = retriever_chunk.retrieve(
    "Can you tell me about the key concepts for safety finetuning"
)
for node in nodes:
    display_source_node(node, source_length=2000)

In [ ]:
query_engine_chunk = RetrieverQueryEngine.from_args(
    retriever_chunk, service_context=service_context
)
response = query_engine_chunk.query(
    "Can you tell me about the key concepts for safety finetuning"
)
print(str(response))

# Evaluation

## QA pair 만들기 (by generate_qa_embedding_pairs)

```
DEFAULT_QA_GENERATE_PROMPT_TMPL = """\
Context information is below.

---------------------
{context_str}
---------------------

Given the context information and not prior knowledge.
generate only questions based on the below query.

You are a Teacher/ Professor. Your task is to setup \
{num_questions_per_chunk} questions for an upcoming \
quiz/examination. The questions should be diverse in nature \
across the document. Restrict the questions to the \
context information provided."
"""

```

In [43]:
from llama_index.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)
import nest_asyncio

nest_asyncio.apply()

In [46]:
base_nodes_sample[0]

TextNode(id_='node-0', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='a7705984-64ba-4a8b-8d84-2196a137671f', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='398cb8798614de7e8a14cfff4b34f87a9f979d4dfaac81358c311ca67a659fe0'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='a3bf5ad9-a072-441c-a9be-7fcf8c4d6894', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='820de41da4f942046105d6d061c52e7e04429023a554953b03b420aaebdb53ae')}, hash='b203c05456cf61d73bf4a69ecb61de212829d2a81f128ed71dc7612ec85ece86', text='Llama 2 : Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗Louis Martin†Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao V

In [ ]:
base_nodes_sample = base_nodes[:5]  # 일부만 사용, 1개의 node context 당 2개의 question 생성
eval_dataset = generate_question_context_pairs(base_nodes_sample,
                                               llm=2,
                                               num_questions_per_chunk=2)  # EmbeddingQAFinetuneDataset (query, relevant_doc)

In [ ]:
eval_dataset.save_json("llama2_eval_dataset.json")

In [ ]:
import pandas as pd
from llama_index.evaluation import RetrieverEvaluator, get_retrieval_results_df

# set vector retriever similarity top k to higher
top_k = 10


def display_results(names, results_arr):
    """Display results from evaluate."""

    hit_rates = []
    mrrs = []
    for name, eval_results in zip(names, results_arr):
        metric_dicts = []
        for eval_result in eval_results:
            metric_dict = eval_result.metric_vals_dict
            metric_dicts.append(metric_dict)
        results_df = pd.DataFrame(metric_dicts)

        hit_rate = results_df["hit_rate"].mean()
        mrr = results_df["mrr"].mean()
        hit_rates.append(hit_rate)
        mrrs.append(mrr)

    final_df = pd.DataFrame(
        {"retrievers": names, "hit_rate": hit_rates, "mrr": mrrs}
    )
    display(final_df)


In [ ]:
# base
base_retriever = base_index.as_retriever(similarity_top_k=top_k)
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=base_retriever
)
results_base = await retriever_evaluator.aevaluate_dataset(
    eval_dataset, show_progress=True
)


In [ ]:
# chunk
vector_retriever_chunk = vector_index_chunk.as_retriever(
    similarity_top_k=top_k
)
retriever_chunk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_chunk},
    node_dict=all_nodes_dict,
    verbose=True,
)
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=retriever_chunk
)

results_chunk = await retriever_evaluator.aevaluate_dataset(
    eval_dataset, show_progress=True
)

In [ ]:
full_results_df = get_retrieval_results_df(
    [
        "Base Retriever",
        "Retriever (Chunk References)"
    ],
    [results_base, results_chunk],
)
display(full_results_df)